# Scraping Tweets from a public Twitter account between two dates

snscraper has a very easy to use python wrapper than can be used to scrape tweets from a public Twitter account.

In [1]:
from tqdm import tqdm
import snscrape.modules.twitter as sntwitter
import pandas as pd
from datetime import date, timedelta
from scrape_utils import scrape_twitter, scrape_twitter_users

In [ ]:
scrape_twitter(query="tropical hut", since="2022-06-01")

In [3]:
df = pd.read_csv("tweets_tropical hut.csv")
df.head()

,tweet_id,date,user_name,tweet,retweets,likes,quote_tweets,replies,retweeted_tweet,quoted_tweet,location
0,1538418137313726466,2022-06-19 07:07:21+00:00,sigarily0,tara tropical hut date🥹,0,1,0,0,NaN,NaN,Baguio City
1,1538418090681470976,2022-06-19 07:07:09+00:00,fckppI,pagkabalik na pagkabalik ko talaga ng manila s...,0,0,0,0,NaN,NaN,NaN
2,1538416292150988801,2022-06-19 07:00:01+00:00,interaksyon,Most of Tropical Hut's stores are located in M...,2,1,0,0,NaN,NaN,Philippines
3,1538414799553368064,2022-06-19 06:54:05+00:00,dumidyeypee,Tropical Hut is now hiring people for deployme...,30,118,5,6,NaN,NaN,NaN
4,1538414257896689665,2022-06-19 06:51:56+00:00,ohhmyGHEE,May Tropical Hut entry din kami 😁🤤💕 https://t....,0,0,0,0,NaN,NaN,NaN


In [ ]:
users = df.user_name.unique().tolist()
scrape_twitter_users(users, filename="tropical hut")

In [11]:
df_users = pd.read_csv("tweets_tropical hut_users.csv")
df_users.head(10)

,user_id,user_name,display_name,description,verified,created,followers,following,tweets,location
0,1247922829372219392,sigarily0,isagani ❁,‧͙⁺˚*･༓arkitektura at tite☽༓･*˚⁺‧͙,False,2020-04-08 16:22:58+00:00,1581,1644,23224,Baguio City
1,1485277679368941568,fckppI,mm,NaN,False,2022-01-23 15:46:36+00:00,107,44,8857,NaN
2,242696106,interaksyon,Interaksyon,Your thoughts on the news. facebook.com/Intera...,True,2011-01-25 11:24:31+00:00,213356,106,200467,Philippines
3,1124692538,dumidyeypee,JP ,Stocks | Numbers | Finance | Data | Politics |...,False,2013-01-27 10:52:27+00:00,8575,3063,162651,NaN
4,94038248,ohhmyGHEE,Ghee.,Jee le zaraa 🌸,False,2009-12-02 06:55:10+00:00,133,480,21884,NaN
5,242775158,frustratedDoe,Singapore Airlines Boarding Music 2021 is ❤️,"Tumbling along life, or IDK.",False,2011-01-25 15:31:51+00:00,808,3552,58743,P.I. (Philippine Islands)
6,120934917,ZutKuy,Zutkuy,Not an alter account but decided to hide my id...,False,2010-03-08 01:33:17+00:00,349,4983,11238,Planet Earth
7,296112902,Idgesheeran,Idge Dizon,2 rekt 2 introspect | UP College of Law | @Roo...,False,2011-05-10 06:50:34+00:00,1164,1004,49741,NaN
8,1428070893410918403,heeluvsyun,seri.,ramyeonz's ramen buddy 🍜,False,2021-08-18 19:07:11+00:00,3112,877,33127,(+63) 04 she/her\nheavy curser ! no priv qrt 🚫...
9,957615504377577473,mglbenedict,miguel,my pronouns are first love/late spring,False,2018-01-28 14:05:02+00:00,126,256,6827,UID: 821038328
